# Introduction to xarray, InferenceData, and NetCDF for ArviZ

While ArviZ supports plotting from familiar datatypes, such as dictionaries and numpy arrays, there are a couple data structures central to ArviZ that are useful to know when using the library. 

They are  

<ul>
  <!-- HTML Table to allow proper rendering in sphinx -->
  <li>xarray.Dataset</li>
  <li>arviz.InferenceData</li>
  <li>netCDF</li>
</ul> 


## Why more than one data structure?
Bayesian Inference generates numerous datasets that represent different aspects of the model. For example in a single analysis a Bayesian practioner could end up with any of the following data.



<ul>
  <!-- HTML Table to allow proper rendering in sphinx -->
  <li>Prior Distribution for N number of variables</li>
  <li>Posterior Distribution for N number of variables</li>
  <li>Prior Predictive Distribution</li>
  <li>Posterior Predictive Distribution</li>
  <li>Trace data for each of the above</li>
  <li>Sample statistics for each inference run</li>
  <li>Any other array like data source</li>

</ul> 

See the `InferenceData` structure specification [here](../schema/schema.html).


## Why not Pandas Dataframes or Numpy Arrays?
Data from probabilistic programming is naturally high dimensional. To add to the complexity ArviZ must handle the data generated from multiple Bayesian Modeling libraries, such as PyMC3 and PyStan. This is an application that the *xarray* package handles quite well. The xarray package lets users manage high dimensional data with human readable dimensions and coordinates quite easily.

![InferenceData Structure](InferenceDataStructure.png)  

Above is a visual representation of the data structures and their relationships. Although seemingly more complex at a glance the ArviZ devs believe that the usage of *xarray*, *InferenceData*, and *NetCDF* will simplify the handling, referencing, and serialization of data generated during Bayesian analysis.  



## An introduction to each
To help get familiar with each, ArviZ includes some toy datasets. To start an `az.InferenceData` sample can be loaded from disk.

In [1]:
# Load the centered eight schools model
import arviz as az

data = az.load_arviz_data("centered_eight")
data

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> observed_data

In this case the `az.InferenceData` object contains both a posterior predictive distribution and the observed data, among other datasets. Each group in InferenceData is both an attribute on `InferenceData` and itself a `xarray.Dataset` object. 

In [2]:
# Get the posterior Dataset
posterior = data.posterior
posterior

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       (chain, draw) float64 -3.477 -2.456 -2.826 ... 4.597 5.899 0.1614
    theta    (chain, draw, school) float64 1.669 -8.537 -2.623 ... 10.59 4.523
    tau      (chain, draw) float64 3.73 2.075 3.703 4.146 ... 8.346 7.711 5.407
Attributes:
    created_at:                 2019-06-21T17:36:34.398087
    inference_library:          pymc3
    inference_library_version:  3.7

In our eight schools example the posterior trace consists of 3 variables, estimated over 4 chains. In addition this model is a hierachial modes where values for the variable `theta` are associated with a particular school. 

In xarray's terminology, data variables are the actual values generated from the MCMC draws, Dimensions are the axes on which refer to the data variables and coordinates which are pointers to specific slices or points in the `xarray.Dataset`

Observed data from the Eight Schools model can be accessed through the same method.

In [3]:
# Get the observed xarray
observed_data = data.observed_data
observed_data

<xarray.Dataset>
Dimensions:  (school: 8)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    obs      (school) float64 28.0 8.0 -3.0 7.0 -1.0 1.0 18.0 12.0
Attributes:
    created_at:                 2019-06-21T17:36:34.491909
    inference_library:          pymc3
    inference_library_version:  3.7

It should be noted that the observed dataset contains only 8 data variables and doesn't have a chain and draw dimension or coordinates unlike posterior. This difference in sizes is the motivating reason behind *InferenceData*. Rather than force multiple different sized arrays into one array, or have users to manage multiple objects corresponding to different datasets, it is easier to hold references to each *xarray.Dataset* in an *InferenceData* object.

## NetCDF
[NetCDF](https://www.unidata.ucar.edu/software/netcdf/) is a standard for referencing array oriented files. In other words while, *xarray.Dataset*s, and by extension *InferenceData*, are convenient for accessing arrays in Python memory, *NetCDF* provides a convenient mechanism for persistence of model data on disk. In fact the NetCDF dataset was the inspiration for *InferenceData* as NetCDF4 supports the concept of groups. *InferenceData* merely wraps xarray.Dataset with the same functionality,

Most users will not have to concern themselves with the *NetCDF* standard but for completeness it is good to make its usage transparent. It is also worth noting that the NetCDF4 file standard is interoperable with HDF5 which may be familiar from other contexts.

Earlier in this tutorial *InferenceData* was loaded from a *NetCDF* file

In [4]:
data = az.load_arviz_data("centered_eight")

Similarly the *InferenceData* objects can be persisted tp disk in the NetCDF format

In [5]:
data.to_netcdf("eight_schools_model.nc")

'eight_schools_model.nc'

# Additional Reading
Additional documentation and tutorials exist for xarray and netcdf4.

## xarray
* [xarray documentation](http://xarray.pydata.org/en/stable/why-xarray.html)
* [xarray lightning talk at scipy 2015](https://www.youtube.com/watch?v=X0pAhJgySxk&t=949s)

## NetCDF
* [NetCDF documentation](http://unidata.github.io/netcdf4-python/)
* [NetCDF usage in xarray](http://xarray.pydata.org/en/stable/io.html#netcdf)
* [NetCDF interoperability with HDF5](https://www.unidata.ucar.edu/software/netcdf/docs/interoperability_hdf5.html)
